# Imports

In [28]:
import re
import csv
import math
import collections

In [29]:
def erase_link(tweet):
    inx = tweet.find('http')
    #Check to see if tweet contains a link
    if inx != -1:
        #Find white space
        sp_inx = tweet.find(" ", inx)
        #Check if there is a white space after the link
        if sp_inx != -1:
            new_tweet = tweet[0:inx] + tweet[sp_inx + 1:]
        else:
            new_tweet = tweet[0:inx]
        return new_tweet
    return tweet

In [30]:
#This function will normalize a 
#tweet passed in as a string. 
#The output will be the text with out
#any special characters in all lowercase.
def normalize(tweet):
    #erase link
    normalizedTweet = erase_link(tweet)
    #print normalizedTweet
    normalizedTweet = re.compile('[^a-zA-Z@]').sub(' ', normalizedTweet)
    normalizedTweet = normalizedTweet.lower()
    normalizedTweet = " ".join(normalizedTweet.split())
    return normalizedTweet

print normalize("123myAPPLE!@Rocks http.bit.ly YES")

myapple @rocks yes


In [31]:
with open("twitter-data/testdata.manual.2009.06.14.csv", 'rb') as csvfile:
    r = csv.reader(csvfile, delimiter=',', quotechar='"')
    docs = [ (x[2], x[4], x[5])  for x in r]
    
tweet_date = [ d[0] for d in docs ] #Tweet dates
tweet_user = [ d[1] for d in docs ] #Tweet users
tweet_text = [ d[2] for d in docs ] #Tweet text

#Normalize each tweet 
tweet_text = map(normalize, tweet_text)


In [32]:
corpus = {}
for x in range(0,len(tweet_text)):
    corpus[x] = [tweet_date[x], tweet_user[x],tweet_text[x]]
    #print corpus[x]



# Inverted Indexs

For the twitter data, there will be two inverted indexes. The first inverted index will have the username as the key and the tweets that correspond to that username as the values. The second inverted index will be the word index for our tweets; the key will be a word and the values will be the id of the documents that contain that word.

# Word Dictionary Reversed Index

The reversed index for the words in the corpus will be created by going through each word in each document. The reveresed index will allow for a faster retrieval of relevant documents. 

In [33]:
def create_inverted_index(corpus):
    idx = {}
    
    for i, doc in enumerate(corpus):
        # Iterate through each word in the document
        for word in doc.split():
            if word in idx:
                # Update the document's term frequency
                if i in idx[word]:
                    idx[word][i] += 1
                # Add the document to the word index
                else:
                    idx[word][i] = 1;
            # Add the word to the reversed index
            else:
                idx[word] = {i:1}
    
    return idx


In [34]:
'''
test_users = ["vcu451", "chadfu", "SIX15"]
test_corpus = ["reading my kindle2  love it lee childs is good read", 
               "ok, first assesment of the kindle2 ...it fucking rocks", 
               "fuck this economy I hate aig and their non loan given asses"]
'''

idx = create_inverted_index(tweet_text)


# User Tweet Index

The user tweet index will allow for the all tweets(documents) that belong to a username to be retrieved quickly.

In [35]:
def create_user_index(users):
    
    user_tweets = {}
    
    # Go through each of the tweets in the corpus
    for i in range( len(users) ):
        # When user already exists, add the document id to the existing user tweet list
        if users[i] in user_tweets:
            user_tweets[users[i] ].append(i)
        # Otherwise, creat a new list with the document id
        else:
            user_tweets[users[i] ] = [i]
            
    return user_tweets

In [36]:
user_tweet_index = create_user_index(tweet_user)

#user_tweet_index

# Document Ranking
To rank the documents, we will be implementing two different ranking algorithms: TF-IDF and BM25. 

# TF-IDF
The TF-IDF (term frequency–inverse document frequency) ranking algorithm ranks documents based on the term frequency of the words in the query in relation to the words in the documents.

In [37]:
def print_results(results, n, head=True):
    ''' Helper function to print results
    '''
    if head:
        print('\nTop %d from recall set of %d items:' % (n, len(results) ) )
        for r in results[:n]:
            print('\t%0.2f - %s' % (r[0], tweet_text[r[1]]))
    else:
        print('\nTop %d from recall set of %d items:' % (n, len(results) ) )
        for r in results[:n]:
            print('\t%0.2f - %s' % (r[0], tweet_text[r[1]]))

In [38]:
def idf(term, idx, n):
    # term - the term that is being scored
    # idx - the reversed index on the terms in the corpus
    # n - the number of docments the term appears in
    return math.log(float(n) / (1 + len(idx[term])))
    
print(idf('how', idx, len(tweet_user)))
print(idf('hate', idx, len(tweet_user)))
print(idf('sleep', idx, len(tweet_user)))
print(idf('whoopi', idx, len(tweet_user)))
print(idf('monkeys', idx, len(tweet_user)))

3.81270480423
3.32022831913
4.60116216459
5.51745289646
5.51745289646


In [39]:
def get_results_tfidf(qry, idx, n):
    score = collections.Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    sorted_results= sorted(results, key=lambda t : t[0] * -1)
    return sorted_results
#results = get_results_tfidf('monkeys', idx, len(tweet_user))
#results = get_results_tfidf('hate', idx, len(tweet_user))
#results = get_results_tfidf('sleep', idx, len(tweet_user))
results = get_results_tfidf('hate', idx, len(tweet_user))

print_results(results, 10)


Top 10 from recall set of 17 items:
	13.28 - on that note i hate word i hate pages i hate latex there i said it i hate latex all you texn rds can come kill me now
	3.32 - i hate revision it s so boring i am totally unprepared for my exam tomorrow things are not looking good
	3.32 - my wrist still hurts i have to get it looked at i hate the dr dentist scary places time to watch eagle eye if you want to join txt
	3.32 - hate safeway select green tea icecream bought two cartons what a waste of money gt lt
	3.32 - fuck this economy i hate aig and their non loan given asses
	3.32 - i hate comcast right now everything is down cable internet amp phone ughh what am i to do
	3.32 - i srsly hate the stupid twitter api timeout thing soooo annoying
	3.32 - son has me looking at cars online i hate car shopping would rather go to the dentist anyone with a good car at a good price to sell
	3.32 - shaunwoo hate n on aig
	3.32 - nooooooo my dvr just died and i was only half way through the ea presser 

# BM25
Implement the BM25 ranking algorithm to rank our results. This ranking algorithm is the one we will be using for the final version of our search engine.

In [40]:
def get_results_bm25(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    
    # n - the length of the corpus
    n = len(corpus)
    
    # d - list with elements corresponding to the length of each document
    d = [len(x.split()) for x in corpus]
    
    # d_avg - the average document length of the docuemnts in the corpus
    d_avg = float(sum(d) / len(d))
    score = collections.Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                # f - the number of times the term appears in the document
                f = float(idx[term][doc])
                # s - the BM25 score for this (term, docuemnt) pair
                s = i * ( (f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc] ) / d_avg) ) ) ) )
                score[doc] += s
                
    results = []
    for x in [ [r[0], r[1] ] for r in zip(score.keys(), score.values() )]:
        if x[1] > 0:
            results.append([ x[1], x[0] ])
            
    sorted_results = sorted(results, key=lambda t: t[0] * -1)
    return sorted_results

results = get_results_bm25('hate', tweet_text, k1=1.5, b=0.75)
#print_results(results, 25)

# Get content based on username
This function will retrieve tweets in the corpus that belong to the specified user. These tweets will be ordered by date. 

In [44]:
def print_results_username(username_index, user_name):
    if(user_name in username_index):
        # All of the documents that correspond to the username 
        docs = username_index[user_name]
        # For every doc, print date, username, and tweet 
        for doc in docs:
            print tweet_date[doc] + " " + tweet_user[doc] + " " + tweet_text[doc]
    else:
        print "Username does not exist"
    
results = create_user_index(tweet_user)
print_results_username(results, 'SimpleManJess')

Tue Jun 02 04:29:16 UTC 2009 SimpleManJess ncaa baseball super regional rams club
Tue Jun 02 04:29:17 UTC 2009 SimpleManJess baseballamerica com blog baseball america prospects blog blog


# Determine User Query
This will take in the user query and determine whether or not the user is wishin to search all tweet content or a specific user's content.

In [41]:
def determine_query(query):
    if(query[0] == '@' and len(query.split()) == 1):
        user_choice = raw_input(("Select your query preference",
                                "\n1. All content",
                                "\n2. User content"))
        return user_choice
    return '1'
    

In [48]:
def u_search(user_choice, query):
    if(user_choice == '1'):
        results = get_results_bm25(query, tweet_text)
        print_results(results,25)
    if(user_choice == '2'):
        results = create_user_index(tweet_user)
        print_results_username(results, query)
    

In [49]:
query = normalize("@kirstiealley")
#choice = determine_query(query)
u_search('1',query)


Top 25 from recall set of 3 items:
	7.38 - @kirstiealley pet dentist
	6.09 - @kirstiealley i hate going to the dentist
	5.60 - @kirstiealley my dentist is great but she s expensive
